In [ ]:
# !pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
# !pip install transformers==4.22.2 datasets==2.4.0 tokenizers==0.12.1

In [ ]:
import pandas as pd
import re
import string
import unicodedata
from tqdm import tqdm

from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("NlpHUST/vi-word-segmentation")
model = AutoModelForTokenClassification.from_pretrained("NlpHUST/vi-word-segmentation")

nlp = pipeline("token-classification", model=model, tokenizer=tokenizer)

In [ ]:
def get_segment(example):
    # Khối try chứa code bạn muốn thực thi
    try:
        # 1. Gọi mô hình NLP (Đây là nơi có khả năng xảy ra lỗi)
        ner_results = nlp(example)
        
        example_tok = ""
        
        # 2. Duyệt qua kết quả và xây dựng chuỗi
        for e in ner_results:
            # Xử lý các token con (sub-words) thường có "##"
            if "##" in e["word"]:
                example_tok = example_tok + e["word"].replace("##","")
            # Xử lý các thực thể tiếp theo trong chuỗi (Thực thể "I" - Inside)
            elif e["entity"] == "I":
                example_tok = example_tok + "_" + e["word"]
            # Xử lý các token khác
            else:
                example_tok = example_tok + " " + e["word"]
        
        # Nếu mọi thứ chạy thành công, trả về chuỗi đã xử lý
        return example_tok.strip() # Dùng .strip() để loại bỏ khoảng trắng thừa ở đầu/cuối
    
    # Khối except sẽ bắt mọi lỗi xảy ra trong khối try
    except Exception as error:
        # Bạn có thể in lỗi ra để gỡ lỗi (debugging) nếu cần
        print(f"Lỗi khi xử lý: {example}. Chi tiết lỗi: {error}")
        
        # Trả về giá trị đầu vào ban đầu theo yêu cầu
        return example

In [ ]:
special_case = {'dđ': 'đ'}
not_in_vietnamese = ['w', 'f', 'j', 'z', 'W', 'F', 'J', 'Z']

trap_in_prompt = [
    'Adversarial Question: ',
    'Noisy Question: ',
    'đây là câu hỏi đã được gài bay: ',
    'câu hỏi gài bẫy có thể như sau: \n\n'
    'câu hỏi gài bẫy có thể là: '
    'câu hỏi gài bẫy: '
    'Câu hỏi gài bẫy: ',
]

INVALID = 'aaaaa'

def remove_diacritics(text: str) -> str:
    nfkd = unicodedata.normalize('NFD', text)
    return ''.join(ch for ch in nfkd if unicodedata.category(ch) != 'Mn')

def replace_word(sentence: str, old: str, new: str) -> str:
    words = sentence.split()
    replaced = [new if w == old else w for w in words]
    return " ".join(replaced)

def modify_word(w):
    """
    Sửa lỗi các từ như "dđ" và các ký tự lặp lại (ví dụ: "haaa" thành "ha").
    Hàm này chỉ xử lý phần chữ cái của từ.
    """
    if len(w) == 0:
        return w
    
    # Xử lý các ký tự lặp lại
    if w[0].islower():
        for c in string.ascii_lowercase:
            while c + c in w:
                w = w.replace(c + c, c)
        for c in not_in_vietnamese:
            w = w.replace(c, '')
    
    # Xử lý các trường hợp đặc biệt
    for old, new in special_case.items():
        if old in w:
            w = w.replace(old, new)
            
    return w

def fix_sentence(s):
    """
    Sửa lỗi từng từ trong một câu trong khi vẫn giữ lại dấu câu.
    """
    # Tìm tất cả các từ (chuỗi chữ cái) và áp dụng hàm modify_word cho chúng
    # Mọi thứ khác (dấu câu, khoảng trắng) sẽ được giữ nguyên
    return re.sub(r'[a-zA-ZÀ-ỹ]+', lambda m: modify_word(m.group(0)), s)

In [ ]:
# df = pd.read_csv('./data/vihallu-train.csv',encoding='utf-8')
# df['prompt'] = df['prompt'].apply(fix_sentence)

In [ ]:
def processing_data(input_file, output_file):
    df = pd.read_csv(input_file,encoding='utf-8')
    df['prompt'] = df['prompt'].apply(fix_sentence)
    
    for i, (c, p, r) in tqdm(enumerate(zip(df['context'], df['prompt'], df['response']))):
        for trap in trap_in_prompt:
            p = p.replace(trap, '').strip()
        
        p = get_segment(p)
        c = get_segment(c)
        r = get_segment(r)
        
        new_response = remove_diacritics(r)
        
        response_dicts = {}
    
        for w, new_w in zip(r.split(' '), new_response.split(' ')):
            if response_dicts.get(new_w) is None:
                response_dicts[new_w] = w
                
            elif response_dicts[new_w] != w:
                response_dicts[new_w] = INVALID
            
        new_context = remove_diacritics(c)
    
        for w, new_w in zip(c.split(' '), new_context.split(' ')):
            if response_dicts.get(new_w) is None:
                response_dicts[new_w] = w
            elif response_dicts[new_w] != w:
                response_dicts[new_w] = INVALID
    
        new_prompt = remove_diacritics(p)
        
        for w, new_w in zip(p.split(' '), new_prompt.split(' ')):
            if response_dicts.get(new_w) is not None and response_dicts.get(new_w) != INVALID:
    
                new_prompt = replace_word(new_prompt, new_w, response_dicts.get(new_w))
            else:
                new_prompt = replace_word(new_prompt, new_w, w)
        df.loc[i, 'prompt'] = new_prompt
    df.to_csv(output_file, index=False, encoding='utf-8')
    print("Exported: ",output_file)

In [ ]:
processing_data('/kaggle/input/dsc25-data/vihallu-train.csv', 'vihallu-train-processed-3.csv')
processing_data('/kaggle/input/dsc25-data/vihallu-public-test.csv', 'vihallu-public-test-processed-3.csv')

In [ ]:
# #In từ điểm ra xem vui vui th
# s = set()

# prompt = df['prompt']
# for p in prompt:
#     #tach cac tu cua p ra roi dua vao set
#     words = p.split(' ')
#     for w in words:
#         s.add(w)

# print(len(s))

# dicts = {}
# for w in s:
#     dicts[w] = 0

# for p in prompt:
#     words = p.split(' ')
    
#     for w in words:
#         if w in dicts:
#             dicts[w] += 1

# dicts = dict(sorted(dicts.items(), key=lambda item: item[1], reverse=True))

# with open('vocab.csv', 'w', encoding='utf-8') as f:
#     for w in dicts:
#         f.write(f'{w},{dicts[w]}\n')